In [13]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

In [52]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Hyper-parameters
sequence_length = 100
input_size = 2
hidden_size = 256
num_layers = 1
batch_size = 100
num_epochs = 5
learning_rate = 0.01

In [53]:
X, y = np.load("train_X.npy"), np.load("train_y.npy")
X = torch.from_numpy(X).float()
y = torch.from_numpy(y).float()

In [54]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.GRU(self.input_size,
                          self.hidden_size,
                          self.num_layers,
                          batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        # Set initial hidden and cell states
        h0 = torch.zeros(self.num_layers, x.size(0),
                         self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0),
                         self.hidden_size).to(device)

        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))  # out: tensor of shape (batch_size, seq_length, hidden_size)

        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        return out

In [55]:
model = RNN(input_size, hidden_size, num_layers).to(device)


In [56]:
# Loss and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [57]:
X, y = X.to(device), y.to(device)

In [58]:
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    
    y_ = model.forward(X)
    loss = criterion(y_, y)
    
    loss.backward(loss)
    optimizer.step()

AttributeError: 'tuple' object has no attribute 'size'